### Making a dataframe that has all the baseball savant percentile rankings

In [3]:
import pybaseball as pb
import pandas as pd
import plotly.express as px
import plotly.io as pio
import numpy as np
from PIL import Image

In [4]:
from pybaseball.statcast_batter import statcast_batter_expected_stats
''' Expected stats '''
# get data for batters with a minimum of 150 plate appearances in 2021
data = statcast_batter_expected_stats(2021, 150)
data

,last_name,first_name,player_id,year,pa,bip,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff
0,Soto,Juan,665742,2021,654,414,0.313,0.304,0.009,0.534,0.544,-0.010,0.420,0.430,-0.010
1,Harper,Bryce,547180,2021,599,360,0.309,0.301,0.008,0.615,0.610,0.005,0.431,0.430,0.001
2,Acuña Jr.,Ronald,660670,2021,360,217,0.283,0.294,-0.011,0.596,0.605,-0.009,0.412,0.427,-0.015
3,Guerrero Jr.,Vladimir,665489,2021,698,496,0.311,0.306,0.005,0.601,0.591,0.010,0.419,0.416,0.003
4,Freeman,Freddie,518692,2021,695,495,0.300,0.319,-0.019,0.503,0.582,-0.079,0.379,0.416,-0.037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,Ahmed,Nick,605113,2021,473,333,0.221,0.208,0.013,0.339,0.290,0.049,0.269,0.250,0.019
324,Mayfield,Jack,608686,2021,290,202,0.218,0.187,0.031,0.387,0.317,0.070,0.284,0.248,0.036
325,Sierra,Magneuris,642423,2021,225,160,0.230,0.226,0.004,0.268,0.263,0.005,0.239,0.239,0.000
326,Fuentes,Joshua,658069,2021,284,207,0.225,0.204,0.021,0.351,0.281,0.070,0.261,0.230,0.031


In [5]:
from pybaseball.statcast_batter import statcast_batter_percentile_ranks

percentiles = statcast_batter_percentile_ranks(2021)
percentiles

,player_name,player_id,year,xwoba,xba,xslg,xiso,xobp,brl,brl_percent,exit_velocity,hard_hit_percent,k_percent,bb_percent,whiff_percent,sprint_speed,oaa
0,Kyle Tucker,663656,2021,95.0,99.0,97.0,92.0,93.0,84.0,76.0,85.0,82.0,81.0,57.0,77.0,60.0,89.0
1,Jean Segura,516416,2021,39.0,70.0,32.0,22.0,53.0,42.0,27.0,16.0,31.0,91.0,23.0,90.0,57.0,95.0
2,Daniel Johnson,669288,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.0,NaN
3,Miguel Andújar,609280,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.0,NaN
4,Mitch Garver,641598,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1052,Logan Webb,657277,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN
1053,Marcus Stroman,573186,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN
1054,José Devers,672701,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.0,NaN
1055,Charlie Morton,450203,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN


#### Turns out there's already a library for getting all the statcast data in one place so don't have to worry about that
#### Only thing I do need to do is limit the PA so need to join with the previous table and to get the PA column

In [6]:
percentiles = pd.merge(percentiles, data[['player_id', 'pa']], on='player_id', how='left')

### May look up sprint speed later but for now only focusing on players with atleast 350 PA(seems like the cut off for making percentiles)

In [7]:
qualified_batters = percentiles[percentiles["pa"] >= 340]

### Now that I have that I want to get their primary position so I can compare others at that posistion as well

In [8]:
from pybaseball.statcast_fielding import statcast_outs_above_average
fielding_df = statcast_outs_above_average(2021, "all", 50)

In [9]:
qualified_batters = pd.merge(qualified_batters, fielding_df[['player_id', 'primary_pos_formatted']], on='player_id', how='left')

In [10]:
qualified_batters.rename(columns={'primary_pos_formatted':'pos'})

,player_name,player_id,year,xwoba,xba,xslg,xiso,xobp,brl,brl_percent,exit_velocity,hard_hit_percent,k_percent,bb_percent,whiff_percent,sprint_speed,oaa,pa,pos
0,Kyle Tucker,663656,2021,95.0,99.0,97.0,92.0,93.0,84.0,76.0,85.0,82.0,81.0,57.0,77.0,60.0,89.0,567.0,RF
1,Jean Segura,516416,2021,39.0,70.0,32.0,22.0,53.0,42.0,27.0,16.0,31.0,91.0,23.0,90.0,57.0,95.0,567.0,SS
2,Guillermo Heredia,628338,2021,12.0,9.0,8.0,14.0,33.0,6.0,13.0,11.0,23.0,36.0,55.0,33.0,67.0,89.0,347.0,CF
3,Raimel Tapia,606132,2021,6.0,47.0,4.0,2.0,24.0,6.0,4.0,5.0,12.0,96.0,30.0,95.0,81.0,82.0,533.0,LF
4,Andrelton Simmons,592743,2021,1.0,18.0,0.0,0.0,9.0,1.0,1.0,1.0,1.0,92.0,26.0,91.0,32.0,99.0,451.0,SS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,César Hernández,514917,2021,30.0,32.0,27.0,30.0,42.0,50.0,33.0,19.0,17.0,51.0,57.0,70.0,89.0,29.0,637.0,2B
228,Pete Alonso,624413,2021,90.0,79.0,93.0,93.0,79.0,98.0,89.0,81.0,81.0,61.0,59.0,45.0,30.0,74.0,637.0,1B
229,Colin Moran,592567,2021,26.0,31.0,24.0,28.0,43.0,23.0,43.0,40.0,27.0,29.0,66.0,35.0,8.0,NaN,359.0,3B
230,Carson Kelly,608348,2021,69.0,53.0,52.0,49.0,83.0,31.0,51.0,32.0,19.0,55.0,87.0,44.0,23.0,NaN,359.0,NaN


### Would be good to have the actual stats next to each percentile too

In [11]:
from pybaseball.statcast_batter import statcast_batter_exitvelo_barrels
exit_velo = statcast_batter_exitvelo_barrels(2021, 200)
exit_velo[exit_velo["last_name"] == 'Soto']

,last_name,first_name,player_id,attempts,avg_hit_angle,anglesweetspotpercent,max_hit_speed,avg_hit_speed,fbld,gb,max_distance,avg_distance,avg_hr_distance,ev95plus,ev95per-swing,ev95percent,barrels,brl_percent,brl_pa
37,Soto,Juan,665742,414,5.8,29.7,116.6,93.0,98.1,89.4,454,149,399,218,23.9,52.7,55.0,13.3,8.4


In [13]:
data[data["last_name"] == 'Soto']

,last_name,first_name,player_id,year,pa,bip,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff
0,Soto,Juan,665742,2021,654,414,0.313,0.304,0.009,0.534,0.544,-0.01,0.42,0.43,-0.01


In [14]:
pd.set_option('display.max_columns', None)
fg_data = pb.batting_stats(2021, qual=200)
fg_data[fg_data["Name"] == 'Juan Soto']

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,BU,BUH,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,wOBA,wRAA,wRC,Bat,Fld,Rep,Pos,RAR,WAR,Dol,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,FB% (Pitch),FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,BsR,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,Def,wSB,UBR,Age Rng,Off,Lg,wGDP,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA
1,20123,2021,Juan Soto,WSN,22,151,502,654,157,106,20,2,29,111,95,145,23,93,2,5,0,23,9,7,0.313,218,119,77,8,2601,1170,1431,13,0,0,0.222,0.142,1.56,0.465,0.534,0.999,0.221,0.332,0.0183,0.186,0.527,0.287,0.067,0.244,0.06,0.0,0.42,57.3,136,52.1,2.2,20.4,-6.5,66.0,6.6,$53.0,3.5,163,4.01,-9.13,13.14,63.67,6.52,0.98,2.06,5,5.06,-0.95,0.488,94.2,0.184,85.0,0.061,88.7,0.094,80.0,0.155,86.2,0.018,86.5,NaN,NaN,0.031,NaN,41.9,3.5,3.5,1.3,2.2,1.3,NaN,3.3,0.74,2.2,0.53,0.55,2.71,NaN,0.151,0.628,0.35,0.626,0.887,0.821,0.416,0.537,0.063,-3.3,0.352,NaN,0.065,0.019,NaN,0.139,0.165,0.086,0.015,NaN,0.152,NaN,NaN,NaN,94.5,NaN,88.9,86.3,NaN,93.4,85.6,79.6,81.4,NaN,86.2,NaN,NaN,-1.8,NaN,0.1,-6.6,NaN,-0.9,0.0,2.3,4.1,NaN,-6.8,NaN,NaN,9.1,NaN,4.7,2.2,NaN,4.8,1.5,-4.7,-6.0,NaN,3.9,NaN,NaN,32.3,NaN,3.4,1.5,NaN,7.0,3.2,-1.1,1.7,NaN,2.0,NaN,NaN,3.53,NaN,2.0,2.93,NaN,1.93,0.74,-0.48,4.39,NaN,0.49,NaN,NaN,0.109,0.588,0.35,0.482,0.879,0.818,0.503,NaN,-4.3,-1.9,0.5,22 - 22,48.8,1.1,-1.9,0.312,0.435,0.254,0.159,0.449,0.391,0.408,0.152,0.001,0.099,0.354,0.058,0.022,NaN,NaN,0.133,0.181,NaN,86.3,74.7,79.2,94.6,88.9,86.7,NaN,NaN,93.5,85.8,NaN,-6.5,8.3,1.7,-1.6,0.6,-6.7,NaN,NaN,-1.1,-0.2,NaN,2.3,-9.4,-6.8,8.0,3.9,1.5,NaN,NaN,3.4,0.0,NaN,1.1,0.1,0.9,31.1,1.9,2.4,NaN,NaN,6.9,4.2,NaN,0.29,3.22,0.36,3.38,1.23,4.1,NaN,NaN,2.01,0.89,NaN,0.115,0.57,0.35,0.49,0.88,0.817,0.516,NaN,NaN,126,241,64,142,127,129,113,0.89,123,79,178,77,124,103,97,89,119,93.0,5.8,55,0.133,116.6,217,0.524,414,0.2,0.263,0.304,0.544,0.43


In [18]:
fielding_df[fielding_df["last_name"] == 'Soto']

,last_name,first_name,player_id,display_team_name,year,primary_pos_formatted,fielding_runs_prevented,outs_above_average,outs_above_average_infront,outs_above_average_lateral_toward3bline,outs_above_average_lateral_toward1bline,outs_above_average_behind,outs_above_average_rhh,outs_above_average_lhh,actual_success_rate_formatted,adj_estimated_success_rate_formatted,diff_success_rate_formatted
340,Soto,Juan,665742,Nationals,NaN,LF,4,4,1,3,-1,0,4,1,90%,89%,1%


### Combining columns from 4 tables

In [23]:
qualified_batters = pd.merge(qualified_batters, exit_velo[['player_id', 'avg_hit_speed', 'max_hit_speed', 'brl_percent']], on='player_id', how='left')

In [25]:
qualified_batters = pd.merge(qualified_batters, data[['player_id', 'woba', 'est_woba', 'est_woba_minus_woba_diff', 'ba', 'est_ba', 'est_ba_minus_ba_diff', 'slg', 'est_slg', 'est_slg_minus_slg_diff']], on='player_id', how='left')

In [28]:
qualified_batters = pd.merge(qualified_batters, fielding_df[['player_id', 'outs_above_average']], on='player_id', how='left')

In [33]:
qualified_batters

,player_name,player_id,year,xwoba,xba,xslg,xiso,xobp,brl,brl_percent_x,exit_velocity,hard_hit_percent,k_percent,bb_percent,whiff_percent,sprint_speed,oaa,pa,primary_pos_formatted,avg_hit_speed,max_hit_speed,brl_percent_y,woba,est_woba,est_woba_minus_woba_diff,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,outs_above_average
0,Kyle Tucker,663656,2021,95.0,99.0,97.0,92.0,93.0,84.0,76.0,85.0,82.0,81.0,57.0,77.0,60.0,89.0,567.0,RF,91.4,111.1,11.6,0.381,0.399,-0.018,0.294,0.307,-0.013,0.557,0.580,-0.023,5.0
1,Jean Segura,516416,2021,39.0,70.0,32.0,22.0,53.0,42.0,27.0,16.0,31.0,91.0,23.0,90.0,57.0,95.0,567.0,SS,87.2,113.1,5.9,0.338,0.318,0.020,0.290,0.265,0.025,0.436,0.394,0.042,8.0
2,Guillermo Heredia,628338,2021,12.0,9.0,8.0,14.0,33.0,6.0,13.0,11.0,23.0,36.0,55.0,33.0,67.0,89.0,347.0,CF,86.6,109.7,4.0,0.292,0.289,0.003,0.220,0.220,0.000,0.354,0.336,0.018,5.0
3,Raimel Tapia,606132,2021,6.0,47.0,4.0,2.0,24.0,6.0,4.0,5.0,12.0,96.0,30.0,95.0,81.0,82.0,533.0,LF,85.7,109.9,2.1,0.305,0.279,0.026,0.273,0.249,0.024,0.372,0.317,0.055,3.0
4,Andrelton Simmons,592743,2021,1.0,18.0,0.0,0.0,9.0,1.0,1.0,1.0,1.0,92.0,26.0,91.0,32.0,99.0,451.0,SS,84.1,108.2,0.6,0.252,0.261,-0.009,0.223,0.230,-0.007,0.274,0.281,-0.007,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,César Hernández,514917,2021,30.0,32.0,27.0,30.0,42.0,50.0,33.0,19.0,17.0,51.0,57.0,70.0,89.0,29.0,637.0,2B,87.4,110.2,6.8,0.302,0.310,-0.008,0.232,0.241,-0.009,0.386,0.384,0.002,-1.0
228,Pete Alonso,624413,2021,90.0,79.0,93.0,93.0,79.0,98.0,89.0,81.0,81.0,61.0,59.0,45.0,30.0,74.0,637.0,1B,91.0,118.4,14.8,0.363,0.376,-0.013,0.262,0.270,-0.008,0.519,0.541,-0.022,1.0
229,Colin Moran,592567,2021,26.0,31.0,24.0,28.0,43.0,23.0,43.0,40.0,27.0,29.0,66.0,35.0,8.0,NaN,359.0,3B,89.1,110.2,8.1,0.318,0.314,0.004,0.258,0.244,0.014,0.390,0.390,0.000,-3.0
230,Carson Kelly,608348,2021,69.0,53.0,52.0,49.0,83.0,31.0,51.0,32.0,19.0,55.0,87.0,44.0,23.0,NaN,359.0,NaN,88.2,109.2,8.5,0.328,0.343,-0.015,0.240,0.251,-0.011,0.411,0.425,-0.014,NaN


In [35]:
''' All the baseball savant data is added - need to do the fangraphs join on Name so have to remove any accents and strip any spaces either side of the name'''
qualified_batters["player_name"] = qualified_batters["player_name"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [36]:
qualified_batters["player_name"] = qualified_batters["player_name"].str.strip()


In [44]:
fg_data.dtypes

IDfg        int64
Season      int64
Name       object
Team       object
Age         int64
           ...   
CStr%     float64
CSW%      float64
xBA       float64
xSLG      float64
xwOBA     float64
Length: 319, dtype: object

In [43]:
qualified_batters.dtypes

player_name                  object
player_id                     int64
year                          int64
xwoba                       float64
xba                         float64
xslg                        float64
xiso                        float64
xobp                        float64
brl                         float64
brl_percent_x               float64
exit_velocity               float64
hard_hit_percent            float64
k_percent                   float64
bb_percent                  float64
whiff_percent               float64
sprint_speed                float64
oaa                         float64
pa                          float64
primary_pos_formatted        object
avg_hit_speed               float64
max_hit_speed               float64
brl_percent_y               float64
woba                        float64
est_woba                    float64
est_woba_minus_woba_diff    float64
ba                          float64
est_ba                      float64
est_ba_minus_ba_diff        

In [45]:
final_df = pd.merge(qualified_batters, fg_data[['Name', 'HardHit%', 'K%', 'BB%', 'SwStr%', 'Spd']], left_on='player_name', right_on='Name', how='left')

In [46]:
final_df

,player_name,player_id,year,xwoba,xba,xslg,xiso,xobp,brl,brl_percent_x,exit_velocity,hard_hit_percent,k_percent,bb_percent,whiff_percent,sprint_speed,oaa,pa,primary_pos_formatted,avg_hit_speed,max_hit_speed,brl_percent_y,woba,est_woba,est_woba_minus_woba_diff,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,outs_above_average,Name,HardHit%,K%,BB%,SwStr%,Spd
0,Kyle Tucker,663656,2021,95.0,99.0,97.0,92.0,93.0,84.0,76.0,85.0,82.0,81.0,57.0,77.0,60.0,89.0,567.0,RF,91.4,111.1,11.6,0.381,0.399,-0.018,0.294,0.307,-0.013,0.557,0.580,-0.023,5.0,Kyle Tucker,0.470,0.159,0.093,0.094,5.3
1,Jean Segura,516416,2021,39.0,70.0,32.0,22.0,53.0,42.0,27.0,16.0,31.0,91.0,23.0,90.0,57.0,95.0,567.0,SS,87.2,113.1,5.9,0.338,0.318,0.020,0.290,0.265,0.025,0.436,0.394,0.042,8.0,Jean Segura,0.374,0.138,0.069,0.077,4.8
2,Guillermo Heredia,628338,2021,12.0,9.0,8.0,14.0,33.0,6.0,13.0,11.0,23.0,36.0,55.0,33.0,67.0,89.0,347.0,CF,86.6,109.7,4.0,0.292,0.289,0.003,0.220,0.220,0.000,0.354,0.336,0.018,5.0,Guillermo Heredia,0.347,0.233,0.092,0.121,2.2
3,Raimel Tapia,606132,2021,6.0,47.0,4.0,2.0,24.0,6.0,4.0,5.0,12.0,96.0,30.0,95.0,81.0,82.0,533.0,LF,85.7,109.9,2.1,0.305,0.279,0.026,0.273,0.249,0.024,0.372,0.317,0.055,3.0,Raimel Tapia,0.315,0.131,0.075,0.066,5.5
4,Andrelton Simmons,592743,2021,1.0,18.0,0.0,0.0,9.0,1.0,1.0,1.0,1.0,92.0,26.0,91.0,32.0,99.0,451.0,SS,84.1,108.2,0.6,0.252,0.261,-0.009,0.223,0.230,-0.007,0.274,0.281,-0.007,16.0,Andrelton Simmons,0.215,0.137,0.071,0.069,2.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,Cesar Hernandez,514917,2021,30.0,32.0,27.0,30.0,42.0,50.0,33.0,19.0,17.0,51.0,57.0,70.0,89.0,29.0,637.0,2B,87.4,110.2,6.8,0.302,0.310,-0.008,0.232,0.241,-0.009,0.386,0.384,0.002,-1.0,Cesar Hernandez,0.331,0.212,0.093,0.092,3.1
228,Pete Alonso,624413,2021,90.0,79.0,93.0,93.0,79.0,98.0,89.0,81.0,81.0,61.0,59.0,45.0,30.0,74.0,637.0,1B,91.0,118.4,14.8,0.363,0.376,-0.013,0.262,0.270,-0.008,0.519,0.541,-0.022,1.0,Pete Alonso,0.473,0.199,0.094,0.115,3.5
229,Colin Moran,592567,2021,26.0,31.0,24.0,28.0,43.0,23.0,43.0,40.0,27.0,29.0,66.0,35.0,8.0,NaN,359.0,3B,89.1,110.2,8.1,0.318,0.314,0.004,0.258,0.244,0.014,0.390,0.390,0.000,-3.0,Colin Moran,0.372,0.242,0.100,0.113,1.5
230,Carson Kelly,608348,2021,69.0,53.0,52.0,49.0,83.0,31.0,51.0,32.0,19.0,55.0,87.0,44.0,23.0,NaN,359.0,NaN,88.2,109.2,8.5,0.328,0.343,-0.015,0.240,0.251,-0.011,0.411,0.425,-0.014,NaN,Carson Kelly,0.336,0.206,0.123,0.099,2.0


In [ ]:
final_df